In [1]:
from __future__ import annotations
import numpy as np
import math
from dataclasses import dataclass, field
from typing import ClassVar, List, Optional, Dict
import pandas as pd

In [42]:
@dataclass
class Bus:
    network: "Network"
    id: Optional[int] = None
    name: Optional[str] = None
    bus_type: str = "PQ"  # "slack", "PQ", "PV"
    v: float = 1.0 # in pu
    theta: float = 0.0 # in degrees
    Sb: float = 1.0 # Base power in MVA
    Sh: float = 0.0 # Shunt admittance connected to the bus

    # Relacionamentos
    loads: List["Load"] = field(default_factory=list)
    generators: List["Generator"] = field(default_factory=list)

    _id_counter: ClassVar[int] = 0

    def __post_init__(self):
        if self.id is None:
            self.id = Bus._id_counter
            Bus._id_counter += 1
        else:
            self.id = int(self.id)
            if self.id >= Bus._id_counter:
                Bus._id_counter = self.id + 1

        if self.name is None:
            self.name = f"Bus {self.id}"

        # Add the bus to the network
        self.network.buses.append(self)
    
    @property
    def theta_rad(self) -> float:
        return np.deg2rad(self.theta)
    
    @property
    def p(self) -> float:
        """Net active power injection (pu)"""
        return sum(g.p for g in self.generators) - sum(l.p for l in self.loads)

    @property
    def q(self) -> float:
        """Net reactive power injection (pu)"""
        return sum(g.q for g in self.generators) - sum(l.q for l in self.loads) 

    @property
    def shunt(self) -> complex:
        """Shunt admittance connected to the bus (pu)"""
        return (self.Sh*1j)/self.Sb
    
    # add_generator and add_load methods are used inside Generator and Load classes automatically. 
    # You just need to inform wich bus the generator or load is connected to.
    def add_generator(self, generator: 'Generator'):
        if generator not in self.generators:
            self.generators.append(generator)

    def add_load(self, load: 'Load'):
        if load not in self.loads:
            self.loads.append(load)

    def __repr__(self):
        return (f"Bus(id={self.id}, type={self.bus_type}, v={self.v:.3f} pu, "
                 f"theta={self.theta_rad:.3f} rad, p={self.p:.3f} pu, q={self.q:.3f} pu, "
                 f"gen={len(self.generators)}, load={len(self.loads)})")
    
@dataclass
class PowerDevice:
    bus: Bus
    name: Optional[str] = None
    pb: float = 1.0
    p_input: float = 0.0
    q_input: float = 0.0
    p_max_input: float = float('inf')
    p_min_input: float = 0.0
    q_max_input: Optional[float] = None
    q_min_input: Optional[float] = None
    cost_a_input: float = 0.0
    cost_b_input: float = 0.0
    cost_c_input: float = 0.0

    _counter: int = field(default=0, init=False, repr=False)

    @property
    def p(self) -> float:
        return self.p_input / self.pb
    
    @property
    def q(self) -> float:
        return self.q_input / self.pb

    @property
    def p_max(self) -> float:
        return self.p_max_input / self.pb
    
    @property
    def p_min(self) -> float:
        return self.p_min_input / self.pb
    
    @property
    def q_max(self) -> Optional[float]:
        return self.q_max_input / self.pb if self.q_max_input is not None else None

    @property
    def q_min(self) -> Optional[float]:
        return self.q_min_input / self.pb if self.q_min_input is not None else None

    @property
    def cost_a(self) -> float:
        return self.cost_a_input * self.pb
    
    @property
    def cost_b(self) -> float:
        return self.cost_b_input * self.pb
    
    @property
    def cost_c(self) -> float:
        return self.cost_c_input * self.pb
    
@dataclass
class Generator(PowerDevice):
    id: Optional[int] = None
    _id_counter: ClassVar[int] = 0
    def __post_init__(self):
        if self.id is None:
            self.id = Generator._id_counter
            Generator._id_counter += 1
        else:
            self.id = int(self.id)
            if self.id >= Generator._id_counter:
                Generator._id_counter = self.id + 1

        if self.name is None:
            self.name = f"Generator {self.id}"
            
        self.bus.add_generator(self)
        self.network = self.bus.network
        self.network.generators.append(self)

    def __repr__(self):
        return (f"Generator(id={self.id}, bus={self.bus.id}, p={self.p:.3f}, q={self.q:.3f}, "
                f"p_range=[{self.p_min:.3f},{self.p_max:.3f}], q_range=[{self.q_min},{self.q_max}])")

@dataclass
class Load(PowerDevice):
    id: Optional[int] = None

    _id_counter: ClassVar[int] = 0

    def __post_init__(self):
        if self.id is None:
            self.id = Load._id_counter
            Load._id_counter += 1
        else:
            self.id = int(self.id)
            if self.id >= Load._id_counter:
                Load._id_counter = self.id + 1

        if self.name is None:
            self.name = f"Load {self.id}"
            
        self.bus.add_load(self)
        self.network = self.bus.network
        self.network.loads.append(self)

    def __repr__(self):
        return (f"Load(id={self.id}, bus={self.bus.id}, p={self.p:.3f}, q={self.q:.3f}, "
                f"p_range=[{self.p_min:.3f},{self.p_max:.3f}], q_range=[{self.q_min},{self.q_max}])")

@dataclass
class Line:
    from_bus: Bus
    to_bus: Bus
    id: Optional[int] = None
    name: Optional[str] = None
    pb: float = 1.0
    vb: float = 1.0
    r: float = 0.0
    x: float = 0.01
    b_half: float = 0.0
    flux_max: float = float('inf')
    tap_ratio: float = 1.0
    tap_phase: float = 0.0

    _id_counter: ClassVar[int] = 0  # ID counter for lines

    def __post_init__(self):
        # Set default values if not provided
        if self.id is None:
            self.id = Line._id_counter
            Line._id_counter += 1
        else:
            self.id = int(self.id)
            if self.id >= Line._id_counter:
                Line._id_counter = self.id + 1

        if self.name is None:
            self.name = f"Line {self.id}"
        else:
            self.name = str(self.name)

        if self.from_bus.network != self.to_bus.network:
            raise ValueError("Both buses must belong to the same network.")
        
        self.network = self.from_bus.network #Add network to line
        self.network.lines.append(self) #Add line to network

    @property
    def zb(self) -> float:
        """Impedância base (pu)"""
        return self.vb**2 / self.pb

    @property
    def resistance(self) -> float:
        """Resistência da linha (pu)"""
        return self.r / self.zb

    @property
    def reactance(self) -> float:
        """Reatância da linha (pu)"""
        return self.x / self.zb

    @property
    def shunt_admittance_half(self) -> float:
        """Admitância shunt (half) (pu)"""
        return self.b_half / self.zb

    @property
    def impedance(self) -> complex:
        """Impedância da linha (ohms)"""
        return complex(self.resistance, self.reactance)

    @property
    def admittance(self) -> complex:
        """Admitância da linha (S)"""
        return 1 / self.impedance

    @property
    def tap_phase_rad(self) -> float:
        """Fase de tap em radianos"""
        return np.deg2rad(self.tap_phase)

    def get_admittance_elements(self, bus_index: Dict[str, int]):
        """Gera os elementos de admitância baseados nos parâmetros da linha"""
        y = self.admittance
        b = self.shunt_admittance_half * 1j
        a = self.tap_ratio * np.exp(1j * self.tap_phase_rad)
        i = bus_index[self.from_bus.id]
        j = bus_index[self.to_bus.id]
        if self.tap_ratio != 1.0 or self.tap_phase != 0.0:
            Yff = y / (a * np.conj(a)) + b
            Yft = -y / np.conj(a)
            Ytf = -y / a
            Ytt = y + b
        else:
            Yff = y + b
            Yft = -y
            Ytf = -y
            Ytt = y + b
        return [((i, i), Yff), ((i, j), Yft), ((j, i), Ytf), ((j, j), Ytt)]

    def __repr__(self):
        return (f"Line(id={self.name}, Barra para:{self.from_bus.id}, Barra de:{self.to_bus.id}, r={self.resistance:.4f}, x={self.reactance:.4f})")
    
@dataclass
class Network:
    id: Optional[int] = None
    name: Optional[str] = None
    buses: List[Bus] = field(default_factory=list)
    lines: List[Line] = field(default_factory=list)
    loads: List[Load] = field(default_factory=list)
    generators: List[Generator] = field(default_factory=list)

    def y_bus(self) -> np.ndarray:
        """
        Returns the Y bus matrix of the network.
        """
        n = len(self.buses)
        bus_idx = {bus.id: i for i, bus in enumerate(self.buses)}
        ybus = np.zeros((n, n), dtype=complex)
        for line in self.lines: #Adiciona os elementos de admitância da linha
            for (i, j), y in line.get_admittance_elements(bus_idx):
                ybus[i, j] += y
        
        for i, bus in enumerate(self.buses):
            ybus[i, i] += bus.shunt
    
        return ybus
        
    def get_G(self):
        return self.y_bus().real
    
    def get_B(self):
        return self.y_bus().imag
    
    def __repr__(self):
        return f"Network(id={self.id}, name={self.name})"


In [43]:
#tap normal
net = Network()

buses = [                                                 
    Bus(net, id= 1, bus_type='Slack', v=1.060, theta=0.0),
    Bus(net, id= 2, bus_type=   'PV', v=1.045, theta=0.0),
    Bus(net, id= 3, bus_type=   'PV', v=1.010, theta=0.0),
    Bus(net, id= 4, bus_type=   'PQ', v=1.000, theta=0.0),
    Bus(net, id= 5, bus_type=   'PQ', v=1.000, theta=0.0),
    Bus(net, id= 6, bus_type=   'PV', v=1.070, theta=0.0),
    Bus(net, id= 7, bus_type=   'PQ', v=1.000, theta=0.0),
    Bus(net, id= 8, bus_type=   'PV', v=1.090, theta=0.0),
    Bus(net, id= 9, bus_type=   'PQ', v=1.000, theta=0.0, Sh=19, Sb=100), ###Faltando shunt!!
    Bus(net, id=10, bus_type=   'PQ', v=1.000, theta=0.0),
    Bus(net, id=11, bus_type=   'PQ', v=1.000, theta=0.0),
    Bus(net, id=12, bus_type=   'PQ', v=1.000, theta=0.0),
    Bus(net, id=13, bus_type=   'PQ', v=1.000, theta=0.0), 
    Bus(net, id=14, bus_type=   'PQ', v=1.000, theta=0.0) 
]

loads = [                                                           
    Load(id= 1, bus=buses[ 1], pb=100, p_input=21.70, q_input=12.70),
    Load(id= 2, bus=buses[ 2], pb=100, p_input=94.20, q_input=19.00),
    Load(id= 3, bus=buses[ 3], pb=100, p_input=47.80, q_input=-3.90),
    Load(id= 4, bus=buses[ 4], pb=100, p_input= 7.60, q_input= 1.60),
    Load(id= 5, bus=buses[ 5], pb=100, p_input=11.20, q_input= 7.50),
    Load(id= 6, bus=buses[ 8], pb=100, p_input=29.50, q_input=16.60),
    Load(id= 7, bus=buses[ 9], pb=100, p_input= 9.00, q_input= 5.80),
    Load(id= 8, bus=buses[10], pb=100, p_input= 3.50, q_input= 1.80),
    Load(id= 9, bus=buses[11], pb=100, p_input= 6.10, q_input= 1.60),
    Load(id=10, bus=buses[12], pb=100, p_input=13.50, q_input= 5.80),
    Load(id=11, bus=buses[13], pb=100, p_input=14.90, q_input= 5.00)
]

#Criação de Geradores:                                        
generators = [                                                
    Generator(id=1, bus=buses[0]),                            
    Generator(id=2, bus=buses[1], pb=100, p_input=40.00),     
    Generator(id=3, bus=buses[2]),                            
    Generator(id=4, bus=buses[4]),                            
    Generator(id=5, bus=buses[5]),                            
    Generator(id=6, bus=buses[7]) 
]

lines = [                                                                                                   
    Line(id= 1, from_bus=buses[ 0], to_bus=buses[ 1], r=0.01938, x=0.05917, b_half=0.0264),                 
    Line(id= 2, from_bus=buses[ 0], to_bus=buses[ 4], r=0.05403, x=0.22304, b_half=0.0246),                 
    Line(id= 3, from_bus=buses[ 1], to_bus=buses[ 2], r=0.04699, x=0.19797, b_half=0.0219),                 
    Line(id= 4, from_bus=buses[ 1], to_bus=buses[ 3], r=0.05811, x=0.17632, b_half=0.0187),                 
    Line(id= 5, from_bus=buses[ 1], to_bus=buses[ 4], r=0.05695, x=0.17388, b_half=0.0170),                 
    Line(id= 6, from_bus=buses[ 2], to_bus=buses[ 3], r=0.06701, x=0.17103, b_half=0.0173),                 
    Line(id= 7, from_bus=buses[ 3], to_bus=buses[ 4], r=0.01335, x=0.04211, b_half=0.0064),                 
    Line(id= 8, from_bus=buses[ 3], to_bus=buses[ 6], r=0.0    , x=0.20912, b_half=0.0    ,tap_ratio=0.978),
    Line(id= 9, from_bus=buses[ 3], to_bus=buses[ 8], r=0.0    , x=0.55618, b_half=0.0    ,tap_ratio=0.969),
    Line(id=10, from_bus=buses[ 4], to_bus=buses[ 5], r=0.0    , x=0.25202, b_half=0.0    ,tap_ratio=0.932),
    Line(id=11, from_bus=buses[ 5], to_bus=buses[10], r=0.09498, x=0.19890, b_half=0.0),                    
    Line(id=12, from_bus=buses[ 5], to_bus=buses[11], r=0.12291, x=0.25581, b_half=0.0),                    
    Line(id=13, from_bus=buses[ 5], to_bus=buses[12], r=0.06615, x=0.13027, b_half=0.0),                    
    Line(id=14, from_bus=buses[ 6], to_bus=buses[ 7], r=0.0    , x=0.17615, b_half=0.0),                    
    Line(id=15, from_bus=buses[ 6], to_bus=buses[ 8], r=0.0    , x=0.11001, b_half=0.0),                    
    Line(id=16, from_bus=buses[ 8], to_bus=buses[ 9], r=0.03181, x=0.08450, b_half=0.0),                    
    Line(id=17, from_bus=buses[ 8], to_bus=buses[13], r=0.12711, x=0.27038, b_half=0.0),                    
    Line(id=18, from_bus=buses[ 9], to_bus=buses[10], r=0.08205, x=0.19207, b_half=0.0),                    
    Line(id=19, from_bus=buses[11], to_bus=buses[12], r=0.22092, x=0.19988, b_half=0.0),                    
    Line(id=20, from_bus=buses[12], to_bus=buses[13], r=0.17093, x=0.34802, b_half=0.0),                    
]

In [44]:
Y = net.y_bus()
Y = pd.DataFrame(Y)
Y

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,6.025029-19.447070j,-4.999132+15.263087j,0.000000+0.000000j,0.000000+ 0.000000j,-1.025897+ 4.234984j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.00000+0.00000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+ 0.000000j,0.000000+0.000000j
1,-4.999132+15.263087j,9.521324-30.270715j,-1.135019+4.781863j,-1.686033+ 5.115838j,-1.701140+ 5.193927j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.00000+0.00000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+ 0.000000j,0.000000+0.000000j
2,0.000000+ 0.000000j,-1.135019+ 4.781863j,3.120995-9.811480j,-1.985976+ 5.068817j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.00000+0.00000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+ 0.000000j,0.000000+0.000000j
3,0.000000+ 0.000000j,-1.686033+ 5.115838j,-1.985976+5.068817j,10.512990-38.635171j,-6.840981+21.578554j,0.000000+ 0.000000j,0.000000+ 4.889513j,0.00000+0.00000j,0.000000+ 1.855500j,0.000000+ 0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+ 0.000000j,0.000000+0.000000j
4,-1.025897+ 4.234984j,-1.701140+ 5.193927j,0.000000+0.000000j,-6.840981+21.578554j,9.568018-35.527539j,0.000000+ 4.257445j,0.000000+ 0.000000j,0.00000+0.00000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+ 0.000000j,0.000000+0.000000j
5,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+0.000000j,0.000000+ 0.000000j,0.000000+ 4.257445j,6.579923-17.340733j,0.000000+ 0.000000j,0.00000+0.00000j,0.000000+ 0.000000j,0.000000+ 0.000000j,-1.955029+4.094074j,-1.525967+3.175964j,-3.098927+ 6.102755j,0.000000+0.000000j
6,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+0.000000j,0.000000+ 4.889513j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000-19.549006j,0.00000+5.67698j,0.000000+ 9.090083j,0.000000+ 0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+ 0.000000j,0.000000+0.000000j
7,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 5.676980j,0.00000-5.67698j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+ 0.000000j,0.000000+0.000000j
8,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+0.000000j,0.000000+ 1.855500j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 9.090083j,0.00000+0.00000j,5.326055-24.092506j,-3.902050+10.365394j,0.000000+0.000000j,0.000000+0.000000j,0.000000+ 0.000000j,-1.424005+3.029050j
9,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.000000+ 0.000000j,0.00000+0.00000j,-3.902050+10.365394j,5.782934-14.768338j,-1.880885+4.402944j,0.000000+0.000000j,0.000000+ 0.000000j,0.000000+0.000000j
